# API Data Pull

In [1]:
# Imports and Connections

import nest_asyncio
nest_asyncio.apply()  # Allow nested event loops in Jupyter notebooks

from ib_insync import *
from pathlib import Path
import pandas as pd

DATA_DIR = Path("../data/raw_ibkr")
DATA_DIR.mkdir(parents=True, exist_ok=True)

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=11)  # use a new clientId for notebooks


<IB connected to 127.0.0.1:7497 clientId=11>

Error 10089, reqId 6: Requested market data requires additional subscription for API. See link in 'Market Data Connections' dialog for more details.SPY ARCA/TOP/ALL, contract: Stock(symbol='SPY', exchange='SMART', currency='USD')
Error 10089, reqId 5: Requested market data requires additional subscription for API. See link in 'Market Data Connections' dialog for more details.Delayed market data is available.AAPL NASDAQ.NMS/TOP/ALL, contract: Stock(symbol='AAPL', exchange='SMART', currency='USD')


In [2]:
# Define helper to fetch 1-min bars

def get_historicals(ticker: str, days: int = 2) -> pd.DataFrame:
    """
    Pull `days` calendar days of 1-minute bars for `ticker`.
    Returns a DataFrame indexed by datetime (UTC).
    """
    contract = Stock(ticker, 'SMART', 'USD')
    bars = ib.reqHistoricalData(
        contract,
        endDateTime='',
        durationStr=f'{days} D',
        barSizeSetting='1 min',
        whatToShow='TRADES',
        useRTH=False,
        formatDate=1
    )
    df = util.df(bars)
    df = df.rename(columns={'date': 'timestamp'}).set_index('timestamp')
    return df


In [3]:
# Pull data & save

symbols = ["AAPL", "SPY"]
for sym in symbols:
    df = get_historicals(sym)
    file_path = DATA_DIR / f"{sym}_1min.parquet"
    df.to_parquet(file_path)
    print(f"Saved {sym}: {df.shape[0]:,} rows →", file_path)


Saved AAPL: 1,807 rows → ..\data\raw_ibkr\AAPL_1min.parquet
Saved SPY: 1,807 rows → ..\data\raw_ibkr\SPY_1min.parquet


In [4]:
# Live quote snapshot via reqMktData()
from time import sleep

symbols = ["AAPL", "SPY"]
tickers = []
for s in symbols:
    contract = Stock(s, 'SMART', 'USD')
    # snapshot=True, regulatorySnapshot=False
    ticker = ib.reqMktData(contract, "", snapshot=True, regulatorySnapshot=False)
    tickers.append(ticker)

sleep(1)  # give IBKR a moment to fill the snapshot

# turn the list of Ticker objects into a DataFrame
live_df = util.df(tickers)
clean_df = live_df.drop(columns=['contract'])
clean_df.to_parquet(DATA_DIR / "snapshots.parquet")
clean_df



,time,marketDataType,minTick,bid,bidSize,bidExchange,ask,askSize,askExchange,last,...,bidGreeks,askGreeks,lastGreeks,modelGreeks,auctionVolume,auctionPrice,auctionImbalance,regulatoryImbalance,bboExchange,snapshotPermissions
0,None,1,NaN,NaN,NaN,,NaN,NaN,,NaN,...,None,None,None,None,NaN,NaN,NaN,NaN,,0
1,None,1,NaN,NaN,NaN,,NaN,NaN,,NaN,...,None,None,None,None,NaN,NaN,NaN,NaN,,0


In [5]:
# Cleanup

ib.disconnect()